In [22]:
## imports
import sys
import os
import numpy as np
import xarray as xr
import cmocean
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import gridspec
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import eofs
from eofs.xarray import Eof
import cftime
import nc_time_axis

import mtspec

# from pylab import *

import datetime
import time
import glob, os
import math
import matplotlib.colors as colors
import xesmf as xe
from defs import regularAtlmask

In [1]:
datafolder = ''

In [23]:
experiment = 'E280' 
model = 'HadCM3'
folder = datafolder+'Raw_data/'+model+'/'

In [24]:
if experiment == 'E280':
    dsT = xr.open_dataset(folder+experiment+'/E280_2900-2999_Annual.nc')
    dsV = xr.open_dataset(folder+experiment+'/E280_2900-2999_Annual.nc')
    dsS = xr.open_dataset(folder+experiment+'/E280_2900-2999_Annual.nc')
elif experiment == 'Eoi400':
    dsT = xr.open_dataset(folder+experiment+'/Eoi400_2400-2499_Annual.nc')
    dsV = xr.open_dataset(folder+experiment+'/Eoi400_2400-2499_Annual.nc')
    dsS = xr.open_dataset(folder+experiment+'/Eoi400_2400-2499_Annual.nc')
dsgrid = xr.open_dataset(folder+experiment+'/'+experiment+'.SST.mean_month.nc')

dsS = dsS.drop(['lat2', 'lat3', 'lat4', 'lon1', 'lon3', 'lon4', 'z1', 'z3', 'z4'])
dsT = dsT.drop(['lat2', 'lat3', 'lat4', 'lon1', 'lon3', 'lon4', 'z1', 'z3', 'z4'])
dsV = dsV.drop(['lat1', 'lat3', 'lat4', 'lon1', 'lon2', 'lon4', 'z1', 'z3', 'z4'])

In [19]:
regridder = xe.Regridder(dsS, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsS.salinity)

ds = xr.Dataset(data_vars = dict(so=(("z","lat","lon"), var)), coords = dict(z = var.z2.values, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/so_1x1_100yr.nc')

In [20]:
regridder = xe.Regridder(dsV, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsV.field704)

ds = xr.Dataset(data_vars = dict(vo=(("z","lat","lon"), var)), coords = dict(z = var.z2.values, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/vo_1x1_100yr.nc')

In [21]:
regridder = xe.Regridder(dsT, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsT.temp_1_opf)

ds = xr.Dataset(data_vars = dict(thetao=(("z","lat","lon"), var)), coords = dict(z = var.z2.values, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/thetao_1x1_100yr.nc')